## 13.4 성능 개선
* EfficientNet b1~3 이용

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
import torch
import random
import os
from tqdm.auto import tqdm

In [26]:
# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [27]:
# GPU 장비 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
data_path = '../../data/13_xray/chest_xray/'

train_path = data_path + 'train/'
valid_path = data_path + 'val/'
test_path = data_path + 'test/'

In [29]:
# 데이터 증강을 위한 이미지 변환기 정의
from torchvision import transforms
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop(180), # 중앙 이미지 확대
    transforms.RandomHorizontalFlip(0.5), # 좌우 대칭
    transforms.RandomVerticalFlip(0.2), # 상하 대칭
    transforms.RandomRotation(20), # 이미지 회전
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
transform_valid = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop(180), # 중앙 이미지 확대
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [30]:
# 데이터셋 및 데이터 로더 생성
from torchvision.datasets import ImageFolder
datasets_train = ImageFolder(root=train_path, transform=transform_train)
datasets_valid = ImageFolder(root=valid_path, transform=transform_valid)

In [31]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

In [32]:
from torch.utils.data import DataLoader

batch_size = 8
loader_train = DataLoader(dataset=datasets_train, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g, num_workers=0)
loader_valid = DataLoader(dataset=datasets_valid, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=0)

### 13.4.1 모델생성

In [33]:
import timm
models_list = []
effiecient_b1 = timm.create_model('efficientnet_b1', pretrained=True).to(device)
effiecient_b2 = timm.create_model('efficientnet_b2', pretrained=True).to(device)
effiecient_b3 = timm.create_model('efficientnet_b3', pretrained=True).to(device)
models_list.append(effiecient_b1)
models_list.append(effiecient_b2)
models_list.append(effiecient_b3)

In [34]:
for idx, model in enumerate(models_list):
    num_params = sum(param.numel() for param in model.parameters())
    print(f'model({idx+1}) 파라미터 개수 " {num_params}')

model(1) 파라미터 개수 " 7794184
model(2) 파라미터 개수 " 9109994
model(3) 파라미터 개수 " 12233232


In [35]:
# 손실 함수와 옵티마이저 설정
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
optimizer1 = torch.optim.Adam(models_list[0].parameters(), lr=0.0006, weight_decay=0.001)
optimizer2 = torch.optim.Adam(models_list[1].parameters(), lr=0.0006, weight_decay=0.001)
optimizer3 = torch.optim.Adam(models_list[2].parameters(), lr=0.0006, weight_decay=0.001)

In [36]:
from transformers import get_cosine_schedule_with_warmup

epochs = 20
scheduler1= get_cosine_schedule_with_warmup(
    optimizer1, 
    num_warmup_steps=len(loader_train)*3,
    num_training_steps=len(loader_train)*epochs
)
scheduler2= get_cosine_schedule_with_warmup(
    optimizer2, 
    num_warmup_steps=len(loader_train)*3,
    num_training_steps=len(loader_train)*epochs
)
scheduler3= get_cosine_schedule_with_warmup(
    optimizer3, 
    num_warmup_steps=len(loader_train)*3,
    num_training_steps=len(loader_train)*epochs
)

In [0]:
### 13.4.2 모델 훈련 및 성능 검증

In [47]:
from sklearn.metrics import accuracy_score,f1_score,recall_score 

In [38]:
# 훈련 함수 작성
def train(model, loader_train, loader_valid, criterion, optimizer, scheduler=None, epochs=10, save_file='model_state_dict.pth'):
    valid_loss_min = np.inf
    for epoch in range(epochs):
        print(f'train epoch [{epoch+1}/{epochs}]\n-------------------------------')
        model.train()
        epoch_train_loss=0
        for images, labels in tqdm(loader_train):
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_train_loss += loss.item()
            loss.backward()
            optimizer.step()
            if scheduler !=None:
                scheduler.step()
        print(f'\t훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
        print(f'valid epoch [{epoch+1}/{epochs}]\n-------------------------------')
        model.eval()
        epoch_valid_loss=0
        preds_list = []
        true_list = []
        with torch.no_grad():
            for images, labels in tqdm(loader_valid):
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_valid_loss += loss.item()
                
                preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
                true = labels.cpu().numpy()
                
                preds_list.extend(preds)
                true_list.extend(true)
                
            print(f'\tvalid 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
            val_accuracy = accuracy_score(true_list, preds_list)
            val_recall = recall_score(true_list, preds_list)
            val_f1_score = f1_score(true_list, preds_list)
            print(f'\t정확도 : {val_accuracy:.4f} / 재현률 : {val_recall:.4f} / F1-score : {val_f1_score:.4f}')
            
            if epoch_valid_loss <= valid_loss_min:
                print(f'\t valid 데이터 loss 감소({valid_loss_min:.4f}=>{epoch_valid_loss:.4f}). 모델 저장')
                torch.save(model.state_dict(), save_file)
                valid_loss_min = epoch_valid_loss
    
    return torch.load(save_file)

In [39]:
# 훈련 및 성능 검증
model_state_dict = train(model=models_list[0],loader_train=loader_train, loader_valid=loader_valid, criterion=criterion, optimizer=optimizer1, scheduler=scheduler1, epochs=epochs)
models_list[0].load_state_dict(model_state_dict)

train epoch [1/20]
-------------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 1.5424
valid epoch [1/20]
-------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

	valid 데이터 손실값 : 1.7000
	정확도 : 0.6250 / 재현률 : 1.0000 / F1-score : 0.7273
	 valid 데이터 loss 감소(inf=>3.3999). 모델 저장
train epoch [2/20]
-------------------------------


...

  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0271
valid epoch [20/20]
-------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

	valid 데이터 손실값 : 0.0673
	정확도 : 1.0000 / 재현률 : 1.0000 / F1-score : 1.0000


<All keys matched successfully>

In [40]:
model_state_dict = train(model=models_list[1],loader_train=loader_train, loader_valid=loader_valid, criterion=criterion, optimizer=optimizer2, scheduler=scheduler2, epochs=epochs)
models_list[1].load_state_dict(model_state_dict)

train epoch [1/20]
-------------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 1.3444
valid epoch [1/20]
-------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

	valid 데이터 손실값 : 0.5653
	정확도 : 0.6875 / 재현률 : 1.0000 / F1-score : 0.7619
	 valid 데이터 loss 감소(inf=>1.1306). 모델 저장
train epoch [2/20]
-------------------------------


...

  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0248
valid epoch [20/20]
-------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

	valid 데이터 손실값 : 0.2303
	정확도 : 0.9375 / 재현률 : 1.0000 / F1-score : 0.9412


<All keys matched successfully>

In [41]:
model_state_dict = train(model=models_list[2],loader_train=loader_train, loader_valid=loader_valid, criterion=criterion, optimizer=optimizer3, scheduler=scheduler3, epochs=epochs)
models_list[2].load_state_dict(model_state_dict)

train epoch [1/20]
-------------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 1.3314
valid epoch [1/20]
-------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

	valid 데이터 손실값 : 0.2833
	정확도 : 0.8750 / 재현률 : 1.0000 / F1-score : 0.8889
	 valid 데이터 loss 감소(inf=>0.5666). 모델 저장
train epoch [2/20]
-------------------------------


...

  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0243
valid epoch [20/20]
-------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

	valid 데이터 손실값 : 0.3021
	정확도 : 0.8750 / 재현률 : 1.0000 / F1-score : 0.8889


<All keys matched successfully>

### 13.4.3 예측 및 평가결과

In [42]:
datasets_test = ImageFolder(root=test_path, transform=transform_valid)
loader_test = DataLoader(
    dataset=datasets_test, 
    batch_size=batch_size, 
    shuffle=False, 
    worker_init_fn=seed_worker,
    generator=g,
    num_workers=0
)

In [43]:
def predict(model, loader_test, return_true=False):
    model.eval()
    preds_list = []
    true_list = []
    
    with torch.no_grad():
        for images, labels in loader_test:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
            true = labels.cpu().numpy()
            preds_list.extend(preds)
            true_list.extend(true)
    if return_true:
        return true_list, preds_list
    else:
        return preds_list

In [44]:
true_list, preds_list1 = predict(model=models_list[0], loader_test=loader_test, return_true=True)
preds_list2 = predict(model=models_list[1], loader_test=loader_test)
preds_list3 = predict(model=models_list[2], loader_test=loader_test)

In [48]:
# 평가 결과
def show_metrics(true_list, preds_list):
    accuracy_ = accuracy_score(true_list, preds_list)
    recall_ = recall_score(true_list, preds_list)
    f1_score_ = f1_score(true_list, preds_list)
    print(f'\t정확도 : {accuracy_:.4f} / 재현률 : {recall_:.4f} / F1-score : {f1_score_:.4f}')

In [49]:
show_metrics(true_list, preds_list1)

	정확도 : 0.8718 / 재현률 : 0.9487 / F1-score : 0.9024


In [50]:
show_metrics(true_list, preds_list2)

	정확도 : 0.9183 / 재현률 : 0.9462 / F1-score : 0.9354


In [51]:
show_metrics(true_list, preds_list3)

	정확도 : 0.8958 / 재현률 : 0.9615 / F1-score : 0.9202


In [52]:
# 앙상블 예측
ensemble_preds = []
for i in range(len(preds_list1)):
    pred_element = np.round((preds_list1[i]+preds_list2[i]+preds_list3[i])/3)
    ensemble_preds.append(pred_element)

In [53]:
# 평가 결과
show_metrics(true_list, ensemble_preds)

	정확도 : 0.9038 / 재현률 : 0.9590 / F1-score : 0.9257
